In [43]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import PowerTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import TransformedTargetRegressor
from sklearn.cluster import KMeans

# import regressors
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor, StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor, PassiveAggressiveRegressor, Perceptron, RidgeClassifier, LogisticRegression
from sklearn.linear_model import Lasso, ElasticNet, Lars, BayesianRidge, ARDRegression, OrthogonalMatchingPursuit, HuberRegressor, TheilSenRegressor, RANSACRegressor
from sklearn.linear_model import LassoLars, LassoLarsIC
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

# pandas deactivate future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

SUBMIT = False
USE_ORIGINAL = False
SEED = 15
SAMPLE = 0.2

train = pd.read_csv('datasets/train.csv')
test = pd.read_csv('datasets/test.csv')
orig = pd.read_csv('datasets/cubic_zirconia.csv')

for i, df in enumerate([train, test, orig]):
    df.drop(['id'], axis=1, inplace=True)
    df.drop_duplicates(inplace=True)
    # df['dataset'] = i

# Define test set
if not SUBMIT:
    train, test = train_test_split(train, test_size=0.2, random_state=SEED) 

if USE_ORIGINAL:
    train = pd.concat([train, orig], axis=0)
    train.reset_index(inplace=True, drop=True)

# Sampling for faster training
if SAMPLE < 1:
    train = train.sample(frac=SAMPLE, random_state=SEED)

del orig

# set training data
X_train = train.copy()
y_train = X_train.pop('price')
X_test = test.copy()

if not SUBMIT:
    y_test = X_test.pop('price')
else:
    y_test = None

# transform categorical features
def transform_categorical(df):
    df['cut'] = df['cut'].map({'Fair': 0, 'Good': 1, 'Very Good': 2, 'Premium': 3, 'Ideal': 4})
    df['color'] = df['color'].map({'J': 0, 'I': 1, 'H': 2, 'G': 3, 'F': 4, 'E': 5, 'D': 6})
    df['clarity'] = df['clarity'].map({'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7})
    return df

def remove_outliers(df):
    # Drop extreme values
    min = 2
    max = 20
    df = df[(df['x'] < max) & (df['y'] < max) & (df['z'] < max)]
    df = df[(df['x'] > min) & (df['y'] > min) & (df['z'] > min)]
    return df

def add_volume(df):
    df['volume'] = df['x'] * df['y'] * df['z']
    return df

def add_volume_ratio(df):
    df['volume_ratio1'] = (df['x'] * df['y']) / (df['z'] * df['z'])
    df['volume_ratio2'] = (df['x'] * df['z']) / (df['y'] * df['y'])
    df['volume_ratio3'] = (df['y'] * df['z']) / (df['x'] * df['x'])
    df['volume_ratio4'] = (df['x'] * df['y']) / (df['z'] * df['y'])
    df['volume_ratio5'] = (df['x'] * df['y']) / (df['z'] * df['x'])
    df['volume_ratio6'] = (df['x'] * df['z']) / (df['y'] * df['x'])
    df['volume_ratio7'] = (df['x'] * df['z']) / (df['y'] * df['z'])
    df['volume_ratio8'] = (df['y'] * df['z']) / (df['x'] * df['y'])
    df['volume_ratio9'] = (df['y'] * df['z']) / (df['x'] * df['z'])
    df['volume_ratio10'] = (df['x'] * df['y']) / (df['y'] * df['z'])
    df['volume_ratio11'] = (df['x'] * df['z']) / (df['x'] * df['y'])
    df['volume_ratio12'] = (df['x'] * df['y']) / (df['x'] * df['z'])
    return df

def add_surface_area(df):
    df['surface_area'] = 2 * (df['x'] * df['y'] + df['x'] * df['z'] + df['y'] * df['z'])
    return df

def feature_engineering(df):
    df["volume"] = df["x"] * df["y"] * df["z"]
    df["surface_area"] = 2 * (df["x"] * df["y"] + df["y"] * df["z"] + df["z"] * df["x"])
    df["aspect_ratio_xy"] = df["x"] / df["y"]
    df["aspect_ratio_yz"] = df["y"] / df["z"]
    df["aspect_ratio_zx"] = df["z"] / df["x"]
    df["diagonal_distance"] = np.sqrt(df["x"] ** 2 + df["y"] ** 2 + df["z"] ** 2)
    df["relative_height"] = (df["z"] - df["z"].min()) / (df["z"].max() - df["z"].min())
    df["relative_position"] = (df["x"] + df["y"] + df["z"]) / (df["x"] + df["y"] + df["z"]).sum()
    df["volume_ratio"] = df["x"] * df["y"] * df["z"] / (df["x"].mean() * df["y"].mean() * df["z"].mean())
    df["length_ratio"] = df["x"] / df["x"].mean()
    df["width_ratio"] = df["y"] / df["y"].mean()
    df["height_ratio"] = df["z"] / df["z"].mean()
    df["sphericity"] = 1.4641 * (6 * df["volume"])**(2/3) / df["surface_area"]
    df["compactness"] = df["volume"]**(1/3) / df["x"]
    return df

def depth_to_table_ratio(df):
    df['depth_to_table_ratio'] = df['depth'] / df['table']
    return df

def new_depth(df):
    df['new_depth'] = df['z'] / (df['x'] + df['y'])
    return df

def target_transform(serie):
    serie = np.log1p(serie)
    return serie

def inverse_target_transform(serie):
    serie = np.expm1(serie)
    return serie

def set_categorical(df):
    df['cut'] = df['cut'].astype('category')
    df['color'] = df['color'].astype('category')
    df['clarity'] = df['clarity'].astype('category')
    return df

# Make data preparation pipeline
def data_prepation(X_train, X_test):
    
    for df in [X_train, X_test]:
        for col in ['x', 'y', 'z']:
            df[col].replace(0, np.nan, inplace=True)
        # df = transform_categorical(df)
        df = set_categorical(df)
    
    # imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    # X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    # X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns, index=X_test.index)
    
    # for df in [X_train, X_test]:
        # df = add_volume(df)
        # df = add_volume_ratio(df)
        # df = add_surface_area(df)
        # df = feature_engineering(df)
        # df = depth_to_table_ratio(df)
        # df = new_depth(df)
        
    # Scaling
    # scaler = PowerTransformer()
    # X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
    # X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
    
    # Clustering features
    # model = KMeans(n_clusters=20, random_state=42)
    # X_train['cluster'] = model.fit_predict(X_train)
    # X_test['cluster'] = model.predict(X_test)
        
    return X_train, X_test
            

X_train_prep, X_test_prep = data_prepation(X_train.copy(), X_test.copy())

In [47]:
cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

# Set categorical features for catboost
cat_features = [col for col in X_train_prep.columns if X_train_prep[col].dtype == 'category']

regressors = {
    'LGBMRegressor1': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='gbdt'),
    # 'LGBMRegressor2': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='dart'),
    # 'LGBMRegressor3': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='goss'),
    # 'LGBMRegressor4': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='rf', subsample=.632, subsample_freq=1),
    # 'LGBMRegressor5': LGBMRegressor(random_state=SEED, n_jobs=-1, class_weight='balanced'),
    # 'LGBMRegressor6': LGBMRegressor(random_state=SEED, n_jobs=-1, subsample=0.7),
    # 'LGBMRegressor7': LGBMRegressor(random_state=SEED, n_jobs=-1, colsample_bytree=0.7),
    # 'LGBMRegressor8': LGBMRegressor(random_state=SEED, n_jobs=-1, subsample=0.7, colsample_bytree=0.7),
    # 'LGBMRegressor9': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='dart', colsample_bytree=0.7),
    # 'XGBRegressor1': XGBRegressor(random_state=SEED, n_jobs=-1),
    # 'XGBRegressor2': XGBRegressor(random_state=SEED, n_jobs=-1, booster='dart'),
    # 'XGBRegressor3': XGBRegressor(random_state=SEED, n_jobs=-1, booster='gblinear'),
    # 'XGBRegressor4': XGBRegressor(random_state=SEED, n_jobs=-1, colsample_bytree=0.7),
    # 'XGBRegressor5': XGBRegressor(random_state=SEED, n_jobs=-1, subsample=0.7),
    # 'XGBRFRegressor6': XGBRegressor(random_state=SEED, n_jobs=-1, objective='reg:squarederror'),
    # 'XGBRandomForestRegressor': XGBRFRegressor(random_state=SEED, n_jobs=-1),
    'CatBoostRegressor': CatBoostRegressor(random_state=SEED, silent=True, cat_features=cat_features), # Promising but fails on the cv
    # 'RandomForestRegressor': RandomForestRegressor(random_state=SEED, n_jobs=-1),
    # 'ExtraTreesRegressor': ExtraTreesRegressor(random_state=SEED, n_jobs=-1),
    # 'AdaBoostRegressor': AdaBoostRegressor(random_state=SEED),
    # 'GradientBoostingRegressor': GradientBoostingRegressor(random_state=SEED),
    # 'BaggingRegressor': BaggingRegressor(random_state=SEED, n_jobs=-1),
    # 'KNeighborsRegressor': KNeighborsRegressor(n_jobs=-1),
    # 'DecisionTreeRegressor': DecisionTreeRegressor(random_state=SEED),
    # 'GaussianProcessRegressor': GaussianProcessRegressor(random_state=SEED),
    # 'MLPRegressor1': MLPRegressor(random_state=SEED, max_iter=1000, activation='relu', solver='adam'),
    # 'MLPRegressor2': MLPRegressor(random_state=SEED, max_iter=1000, activation='relu', solver='lbfgs'), # promising but long to train
    # 'MLPRegressor3': MLPRegressor(random_state=SEED, max_iter=5000, activation='tanh', solver='adam'),
    # 'MLPRegressor4': MLPRegressor(random_state=SEED, max_iter=1000, activation='tanh', solver='lbfgs'),  # promising but long to train
    # 'MLPRegressor5': MLPRegressor(random_state=SEED, max_iter=1000, activation='logistic', solver='adam'),
    # 'MLPRegressor6': MLPRegressor(random_state=SEED, max_iter=1000, activation='logistic', solver='lbfgs'),
    # 'MLPRegressor7': MLPRegressor(random_state=SEED, max_iter=1000, activation='identity', solver='adam'),
    # 'MLPRegressor8': MLPRegressor(random_state=SEED, max_iter=1000, activation='identity', solver='lbfgs'),
    # 'Ridge': Ridge(random_state=SEED),
    # 'SGDRegressor': SGDRegressor(random_state=SEED, max_iter=1000, tol=1e-3),
    # 'PassiveAggressiveRegressor': PassiveAggressiveRegressor(random_state=SEED, max_iter=1000, tol=1e-3),
    # 'Perceptron': Perceptron(random_state=SEED, max_iter=1000, tol=1e-3),
    # 'LinearRegression': LinearRegression(),
    # 'Lasso': Lasso(random_state=SEED),
    # 'ElasticNet': ElasticNet(random_state=SEED, max_iter=1e6),
    # 'HuberRegressor': HuberRegressor(max_iter=1000),
    # 'BayesianRidge': BayesianRidge(),
    # 'ARDRegression': ARDRegression(),
    # 'TheilSenRegressor': TheilSenRegressor(random_state=SEED),
    # 'RANSACRegressor': RANSACRegressor(random_state=SEED),
    # 'OrthogonalMatchingPursuit': OrthogonalMatchingPursuit(normalize=False),
    # 'Lars': Lars(),
    # 'LassoLars': LassoLars(),
    # 'LassoLarsIC': LassoLarsIC(normalize=False),
    # 'StackingRegressor': StackingRegressor(
    #         estimators=[
    #             ('LGBMRandomForestRegressor', LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='rf', subsample=.632, subsample_freq=1)),
    #             ('XGBRandomForestRegressor', XGBRFRegressor(random_state=SEED, n_jobs=-1)),
    #             ('RandomForestRegressor', RandomForestRegressor(random_state=SEED, n_jobs=-1)),
    #             # ('ExtraTreesRegressor', ExtraTreesRegressor(random_state=SEED, n_jobs=-1))
    #             ], 
    #         final_estimator=Ridge(random_state=SEED),
    #         # cv=cv,
    #         # n_jobs=-1,
    #         )
}

for model_name, regressor in regressors.items():
    t0 = time.time()
    scores = []
    feature_importances = pd.DataFrame()
    
    for i, (train_index, test_index) in tqdm(enumerate(cv.split(X_train))):
        
        X_train_cv, X_test_cv = X_train.iloc[train_index].copy(), X_train.iloc[test_index].copy()
        y_train_cv, y_test_cv = y_train.iloc[train_index].copy(), y_train.iloc[test_index].copy()
        X_train_cv, X_test_cv = data_prepation(X_train_cv, X_test_cv)
        
        ttr = TransformedTargetRegressor(regressor=regressor, func=target_transform, inverse_func=inverse_target_transform, check_inverse=False)
        ttr.fit(X_train_cv, y_train_cv)
        
        y_pred = ttr.predict(X_test_cv)
        score_eval = mean_squared_error(y_test_cv, y_pred, squared=False)
        scores.append(score_eval)
        # print(f'{model_name} {i}: {score_eval:.4f}')
        
        try:
            feature_importance = pd.Series(ttr.regressor_.feature_importances_, index=X_train_cv.columns, name=f'fold{i}')
        except:
            feature_importance = pd.Series(ttr.regressor_.coef_, index=X_train_cv.columns, name=f'fold{i}')
            
        # print(feature_importance.sort_values('importance', ascending=False).head(10))
        feature_importances = pd.concat([feature_importances, feature_importance], axis=1)
    
    feature_importances['mean'] = feature_importances.mean(axis=1)
    
    ttr = TransformedTargetRegressor(regressor=regressor, func=target_transform, inverse_func=inverse_target_transform, check_inverse=False)
    ttr.fit(X_train_prep, y_train)
    y_pred = ttr.predict(X_test_prep)
    
    if not SUBMIT:
        score_eval = mean_squared_error(y_test, y_pred, squared=False)
    
    print(f'{model_name}: {np.mean(scores):.4f} ± {np.std(scores):.4f}, Time: {time.time() - t0:.2f} seconds, RMSE: {score_eval:.4f}')
    print(feature_importances.sort_values('mean', ascending=False).head(10))



5it [00:04,  1.05it/s]


LGBMRegressor1: 610.7947 ± 11.9587, Time: 5.98 seconds, RMSE: 587.4274
         fold0  fold1  fold2  fold3  fold4   mean
clarity    575    579    586    574    586  580.0
color      495    470    481    487    511  488.8
y          439    415    446    447    415  432.4
x          430    446    410    399    418  420.6
carat      307    309    328    323    341  321.6
z          306    302    291    277    278  290.8
depth      255    275    266    287    265  269.6
table      124    121    116    132    104  119.4
cut         69     83     76     74     82   76.8


5it [08:33, 102.68s/it]


CatBoostRegressor: 598.6884 ± 15.6859, Time: 579.28 seconds, RMSE: 576.5723
             fold0      fold1      fold2      fold3      fold4       mean
carat    20.777572  21.626585  22.895891  21.566513  24.737009  22.320714
y        19.141864  22.367162  21.965191  23.317676  23.126413  21.983661
x        22.024058  22.229469  20.475241  19.984907  19.126909  20.768117
z        21.171992  17.499012  17.483887  18.131388  16.312701  18.119796
clarity  10.074438   9.741946  10.526698  10.162328  10.002611  10.101604
color     5.988140   5.680606   5.907946   6.103338   5.972845   5.930575
cut       0.364745   0.362329   0.355344   0.320201   0.348076   0.350139
depth     0.272214   0.309535   0.246786   0.251495   0.257523   0.267511
table     0.184976   0.183358   0.143016   0.162154   0.115913   0.157883


In [45]:
X_train_prep.dtypes

carat       float64
cut        category
color      category
clarity    category
depth       float64
table       float64
x           float64
y           float64
z           float64
dtype: object

In [ ]:
# y_pred_test = test_predictions.mean(axis=1).round().astype(int)

# sub = pd.read_csv('submissions/sample_submission.csv')
# sub['quality'] = y_pred_test
# now = time.strftime("%Y-%m-%d %H_%M_%S")
# sub.to_csv(f'submissions/submission{now}.csv', index=False)
# # Copy the leaked values from the original dataset before submitting
# # Transform the price column back to the original scale